In [1]:
import cv2
from glob import glob
from pathlib import Path

from utils import plot
import torch
import torchvision.transforms as transforms

In [2]:
model = torch.load('clip_model_v2.pt')
model.model.eval()

CLIP(
  (visual): VisionTransformer(
    (conv1): Conv2d(3, 768, kernel_size=(32, 32), stride=(32, 32), bias=False)
    (ln_pre): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
    (transformer): Transformer(
      (resblocks): Sequential(
        (0): ResidualAttentionBlock(
          (attn): MultiheadAttention(
            (out_proj): _LinearWithBias(in_features=768, out_features=768, bias=True)
          )
          (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
          (mlp): Sequential(
            (c_fc): Linear(in_features=768, out_features=3072, bias=True)
            (gelu): QuickGELU()
            (c_proj): Linear(in_features=3072, out_features=768, bias=True)
          )
          (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        )
        (1): ResidualAttentionBlock(
          (attn): MultiheadAttention(
            (out_proj): _LinearWithBias(in_features=768, out_features=768, bias=True)
          )
          (ln_1): LayerNorm((7

In [1]:
import cv2
from PIL import Image
import torch

model = torch.load('clip_model_v2.pt')
model.model.eval()

# Initialize the camera
cap = cv2.VideoCapture(0)

# Check if the webcam is opened correctly
if not cap.isOpened():
    raise IOError("Cannot open webcam")

# Load the CLIP model and processor
frame_count = 0

predicted_label = "dadada"

while True:
    ret, frame = cap.read()  # Capture frame-by-frame
    if not ret:
        print("Failed to grab frame")
        break
    
    
    # Process every 30th frame or so for analysis; adjust the frequency as needed
    # if frame_count % 30 == 0:
    #     predicted_label = model.predict(frame)
    #     cv2.putText(frame, {predicted_label},(50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2, cv2.LINE_AA)
    # predicted_label = model.predict(frame)
    # print(predicted_label)

    predicted_label = model.predict(frame)['label']
    cv2.putText(frame, predicted_label,(50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2, cv2.LINE_AA)
        
    cv2.imshow('Video', frame)
    frame_count += 1
    # Break the loop with the 'q' key
    if cv2.waitKey(1) == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

In [4]:
import cv2
from PIL import Image
import torch

# Load your custom model (make sure it's in evaluation mode if necessary)
model = torch.load('clip_model_v2.pt')

# Initialize the camera
cap = cv2.VideoCapture(0)
if not cap.isOpened():
    raise IOError("Cannot open webcam")

# Prediction settings
prediction_threshold = 0.23
danger_labels = set(['fight on a street', 'fire on a street', 'street violence','car crash','fire in office'])  # Add more as needed

# For alerting
alert_count_threshold = 60  # Number of consecutive frames a danger label must be detected to trigger an alert
current_alert_label = None
current_alert_count = 0

while True:
    ret, frame = cap.read()
    if not ret:
        print("Failed to grab frame")
        break

    # Assuming model.predict returns a dictionary with 'label' and 'confidence'
    prediction = model.predict(frame)  # This needs to be defined to work with your model
    label, confidence = prediction['label'], prediction['confidence']

    # Check if the prediction is above the threshold and is a danger label
    if confidence >= prediction_threshold and label in danger_labels:
        if label == current_alert_label:
            current_alert_count += 1
        else:
            current_alert_label = label
            current_alert_count = 1

        if current_alert_count >= alert_count_threshold:
            print(f"ALERT: {label} detected consistently. Take action!")
            # Additional alerting logic can be added here (e.g., sending notifications)
    else:
        current_alert_label = None
        current_alert_count = 0

    # Optionally display the prediction on the frame
    cv2.putText(frame, f"{label}: {confidence:.2f}", (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2, cv2.LINE_AA)

    cv2.imshow('Video', frame)
    
    if cv2.waitKey(1) == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

ALERT: fire in office detected consistently. Take action!
ALERT: fire in office detected consistently. Take action!
ALERT: fire in office detected consistently. Take action!
ALERT: fire in office detected consistently. Take action!
ALERT: fire in office detected consistently. Take action!
ALERT: fire in office detected consistently. Take action!
ALERT: fire in office detected consistently. Take action!
ALERT: fire in office detected consistently. Take action!
ALERT: fire in office detected consistently. Take action!
ALERT: fire in office detected consistently. Take action!
ALERT: fire in office detected consistently. Take action!
ALERT: fire in office detected consistently. Take action!
ALERT: fire in office detected consistently. Take action!
ALERT: fire in office detected consistently. Take action!
ALERT: fire in office detected consistently. Take action!
ALERT: fire in office detected consistently. Take action!
ALERT: fire in office detected consistently. Take action!
ALERT: fire in

In [1]:
import cv2
from PIL import Image
import torch

# Assuming model loading and initialization as before
model = torch.load('clip_model_v2.pt')
model.model.eval()

danger_labels = set(['fight on a street', 'fire on a street', 'street violence','car crash','fire in office','violence in office','fire','violence','accident'])

label_mapping = {
    'fire on a street': 'fire',
    'fire in office': 'fire',
    'fight on a street': 'violence',
    'street violence': 'violence',
    'car crash': 'accident',
}

cap = cv2.VideoCapture(0)

if not cap.isOpened():
    raise IOError("Cannot open webcam")

frame_window_size = 25  # Number of frames in a 3-second window at 30 fps
alert_threshold = 19  # Minimum number of frames with the same class to trigger an alert
recent_predictions = []  # Initialize a list to keep track of recent predictions

while True:
    ret, frame = cap.read()
    if not ret:
        print("Failed to grab frame")
        break

    # Directly using model.predict() as indicated
    predicted_label = model.predict(frame)['label']  # Replace this with your actual model's prediction method if necessary

    # Add the predicted label to the recent predictions list
    if predicted_label in  danger_labels:
        predicted_label = label_mapping.get(predicted_label, predicted_label)
    recent_predictions.append(predicted_label)

    if len(recent_predictions) > frame_window_size:
        recent_predictions.pop(0)  

    # Check if any label meets the alert criteria
    if len(recent_predictions) >= frame_window_size:
        most_common_label = max(set(recent_predictions), key=recent_predictions.count)
        count_most_common = recent_predictions.count(most_common_label)

        if count_most_common >= alert_threshold and most_common_label in danger_labels:
            print(f"ALERT: {most_common_label} detected consistently in the last 3 seconds.")
            # Option to clear the list for reset after alerting
            # recent_predictions.clear()

    cv2.putText(frame, f"{predicted_label}", (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2, cv2.LINE_AA)

    cv2.imshow('Video Feed', frame)

    # Break out of the loop on pressing 'q'
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

Failed to grab frame


In [1]:
import cv2
import torch
import time

# Assuming model loading and initialization as before
model = torch.load('clip_model_v2.pt')
model.model.eval()

danger_labels = set(['fight on a street', 'fire on a street', 'street violence', 'car crash', 'fire in office', 'violence in office', 'fire', 'violence', 'accident'])

label_mapping = {
    'fire on a street': 'fire',
    'fire in office': 'fire',
    'fight on a street': 'violence',
    'street violence': 'violence',
    'car crash': 'accident',
}

cap = cv2.VideoCapture(0)

if not cap.isOpened():
    raise IOError("Cannot open webcam")

frame_window_size = 25  # Number of frames in a 3-second window at 30 fps
alert_threshold = 19  # Minimum number of frames with the same class to trigger an alert
cooldown_period = 5 * 60  # 5 minutes cooldown period in seconds
alert_display_duration = 3  # Duration to display alert message in seconds

recent_predictions = []  # Initialize a list to keep track of recent predictions
last_alert_time = 0  # Time of the last alert
display_alert_until = 0  # Time until which to display the alert message

while True:
    ret, frame = cap.read()
    if not frame:
        print("Failed to grab frame")
        break

    predicted_label = model.predict(frame)['label']  # Replace this with your actual model's prediction method if necessary

    if predicted_label in danger_labels:
        predicted_label = label_mapping.get(predicted_label, predicted_label)
    recent_predictions.append(predicted_label)

    if len(recent_predictions) > frame_window_size:
        recent_predictions.pop(0)

    current_time = time.time()

    if len(recent_predictions) >= frame_window_size and (current_time - last_alert_time) > cooldown_period:
        most_common_label = max(set(recent_predictions), key=recent_predictions.count)
        count_most_common = recent_predictions.count(most_common_label)

        if count_most_common >= alert_threshold and most_common_label in danger_labels:
            print(f"ALERT: {most_common_label} detected consistently in the last 3 seconds.")
            last_alert_time = current_time
            display_alert_until = current_time + alert_display_duration
            # Clear recent_predictions to reset detection after alerting
            recent_predictions.clear()

    # Display the predicted label or alert message
    if current_time <= display_alert_until:
        # Display alert message in red text
        cv2.putText(frame, f"{most_common_label} Alert", (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2, cv2.LINE_AA)
    else:
        # Display the current prediction in green text
        cv2.putText(frame, f"{predicted_label}", (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2, cv2.LINE_AA)

    cv2.imshow('Video Feed', frame)

    # Break out of the loop on pressing 'q'
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


Failed to grab frame


In [2]:
from firebase_admin import credentials
import firebase_admin

cred = credentials.Certificate('./serviceAccountKey.json')
firebase_admin.initialize_app(cred)


In [9]:
#firebase alert
from io import BytesIO
import firebase_admin
from firebase_admin import credentials
import pytz
from datetime import datetime
from firebase_admin import db
from firebase_admin import storage, firestore
import smtplib


# def upload_frame_to_firebase(frame, file_path):
#     # Convert the frame to JPEG format
#     _, buffer = cv2.imencode('.jpg', frame)
#     image_data = BytesIO(buffer)
    
#     # Create a reference to the upload path in Firebase Storage
#     bucket = storage.bucket('smart-threat-detector.appspot.com')
#     blob = bucket.blob(file_path)
    
#     # Upload the image
#     blob.upload_from_file(image_data, content_type='image/jpeg')
    
#     # Make the blob publicly viewable
#     blob.make_public()
    
#     print(f"Uploaded frame to {file_path}")
#     return blob.public_url

# def add_alert_to_database(image_url, location):
#     # Get the current time in UTC
#     utc_now = datetime.now(pytz.UTC)
    
#     # Format the data to write to the database
#     data = {
#         'image_url': image_url,
#         'location': location,
#         'time': utc_now.isoformat()
#     }
    
#     # Create a unique key for the new alert entry
#     ref = db.reference('/alerts')
#     new_alert_ref = ref.push()
    
#     # Write the data to the new alert entry
#     new_alert_ref.set(data)
    
#     print(f"Alert added to database with URL: {image_url}")

#define threat squad for Fire, Accident, Violence in a map 
threat_squad = {
    'fire': ['Fire Station', 'Police', 'Medical'],
    'accident': ['Police', 'Medical'],
    'violence': ['Police']
}

def add_alert_to_database(file_path,frame, location,alert_type):
    # Convert the frame to JPEG format
    _, buffer = cv2.imencode('.jpg', frame)
    image_data = BytesIO(buffer)
    
    # Create a reference to the upload path in Firebase Storage
    bucket = storage.bucket('godsend-aef97.appspot.com')
    blob = bucket.blob(file_path)
    
    # Upload the image
    blob.upload_from_file(image_data, content_type='image/jpeg')
    
    # Make the blob publicly viewable
    blob.make_public()
    
    print(f"Uploaded frame to {file_path}")

    image_url = blob.public_url
    # Get the current time in UTC
    utc_now = datetime.now(pytz.UTC)
    
    # Format the data to write to the database
    data = {
        'cordinates': firestore.GeoPoint(37.422, -122.084),
        'address': 'NMAMIT, Nitte, Karkala, Karnataka, India',
        "image_link": [image_url],
        'squads': threat_squad[alert_type],
        "threat": alert_type,
        'timeStamp': utc_now.isoformat()
    }

    db = firestore.client()

    # Add a new document
    db.collection(location).document().set(data)

#create a mail send notification which fetch admin detail from database "sqaud_users" have a field "email" notify all the admin of category in "category" field defines the threat_squad
# async def send_mail_notification(location,alert_type):
#     db = firestore.client()
#     # Get all the admin emails for the corresponding category of squad
#     admin_emails = db.collection('squad_users').where('category', 'in', threat_squad[alert_type]).stream()
    
#     for admin in admin_emails:
#         email = admin.to_dict()['email']

        
        

#create alert function to add in database and send mail alert make 
async def create_alert(frame, location,alert_type):
    # Save the frame to Firebase Storage
    file_path = f"frames/{location}/alert_{datetime.now().isoformat()}.jpg"
    await add_alert_to_database(file_path,frame, location,alert_type)

    # Send email notifications to the corresponding squad members

    
    
    
    # Additional alerting logic can be added here (e.g., sending notifications)

In [11]:
import cv2
import torch
import time
from datetime import datetime
import asyncio

# Assuming model loading and initialization as before
model = torch.load('clip_model_v2.pt')
model.model.eval()

danger_labels = set(['fight on a street', 'fire on a street', 'street violence', 'car crash', 'fire in office', 'violence in office', 'fire', 'violence', 'accident'])

label_mapping = {
    'fire on a street': 'fire',
    'fire in office': 'fire',
    'fight on a street': 'violence',
    'street violence': 'violence',
    'car crash': 'accident',
}

video_path = './fire_cctv.mp4'

cap = cv2.VideoCapture(video_path)

if not cap.isOpened():
    raise IOError("Cannot open webcam")

frame_window_size = 25  # Number of frames in a 3-second window at 30 fps
alert_threshold = 19  # Minimum number of frames with the same class to trigger an alert
cooldown_period = 5 * 60  # 5 minutes cooldown period in seconds
alert_display_duration = 3  # Duration to display alert message in seconds

recent_predictions = []  # Initialize a list to keep track of recent predictions
last_alert_time = 0  # Time of the last alert
display_alert_until = 0  # Time until which to display the alert message

while True:
    ret, frame = cap.read()
    if not ret:
        print("Failed to grab frame")
        break

    predicted_label = model.predict(frame)['label']  # Replace this with your actual model's prediction method if necessary

    if predicted_label in danger_labels:
        predicted_label = label_mapping.get(predicted_label, predicted_label)
    recent_predictions.append(predicted_label)

    if len(recent_predictions) > frame_window_size:
        recent_predictions.pop(0)

    current_time = time.time()

    if len(recent_predictions) >= frame_window_size and (current_time - last_alert_time) > cooldown_period:
        most_common_label = max(set(recent_predictions), key=recent_predictions.count)
        count_most_common = recent_predictions.count(most_common_label)

        if count_most_common >= alert_threshold and most_common_label in danger_labels:
            print(f"ALERT: {most_common_label} detected consistently in the last 3 seconds.")
            last_alert_time = current_time
            display_alert_until = current_time + alert_display_duration
            # Clear recent_predictions to reset detection after alerting
            recent_predictions.clear()

    # Display the predicted label or alert message
    if current_time <= display_alert_until:
        # Display alert message in red text
        cv2.putText(frame, f"{most_common_label} Alert", (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2, cv2.LINE_AA)
        timestamp = datetime.now().timestamp()
        file_path = f"alerts/{most_common_label}_{timestamp}.jpg"
        add_alert_to_database(file_path,frame, "Karkala",most_common_label)
    else:
        # Display the current prediction in green text
        if predicted_label in danger_labels:
            cv2.putText(frame, f"{predicted_label}", (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2, cv2.LINE_AA)
        else:
            cv2.putText(frame, "Normal", (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2, cv2.LINE_AA)
        

    cv2.imshow('Video Feed', frame)

    # Break out of the loop on pressing 'q'
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


ALERT: fire detected consistently in the last 3 seconds.
Uploaded frame to alerts/fire_1711094780.271099.jpg
